In [162]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import accuracy_score,f1_score,precision_score, recall_score,confusion_matrix
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
df  = pd.read_csv('spam.csv',encoding = "ISO-8859-1")
df.columns=['class','sms','2','3','4']
df.drop(columns=['2','3','4'],inplace=True)
df
corpus = []
train_corpus=[]
test_corpus=[]

In [163]:
labelenc = LabelEncoder()

geo_enc = labelenc.fit_transform(df['class'].unique())
geo_dict = {i:j for i,j in zip(df['class'].unique(),geo_enc)}
df['class'] = df['class'].map(geo_dict)



In [164]:
for i in df['sms']:
    corpus.append(i)
corpus = np.array(corpus)
df,dtest = train_test_split(df)
for i in df['sms']:
    train_corpus.append(i)
train_corpus = np.array(train_corpus)

for i in dtest['sms']:
    test_corpus.append(i)
test_corpus = np.array(test_corpus)


In [165]:
words=[]
train_words=[]
test_words = []
for i in corpus:
  words.append(simple_preprocess(i))
for i in train_corpus:
  train_words.append(simple_preprocess(i))
for i in test_corpus:
  test_words.append(simple_preprocess(i))

In [ ]:
words

In [166]:
model = Word2Vec(words,window=10,min_count=1)
model.train(words, total_examples=len(words), epochs=10)


(644675, 782790)

In [167]:
dict  = {i:j for i,j in zip(model.wv.index_to_key,model.wv.get_normed_vectors())}


In [168]:
def calcavg(X):
    avg = np.zeros(len(dict['go']))
    count = 0
    for i in X:
        avg+=dict[i]
        count+=1
    if count!=0:
        avg/=count
    return avg

In [171]:
avgs = []
for i in train_words:
    avgs.append(calcavg(i))
test_avgs = []
for i in test_words:
    test_avgs.append(calcavg(i))

In [ ]:
avgs

In [172]:
predictor = LogisticRegression()
predictor.fit(avgs,df['class'])

LogisticRegression()

In [173]:
pred  = predictor.predict(test_avgs)
accuracy_score(dtest['class'],pred)

0.9676956209619526

In [174]:
pred  = predictor.predict(avgs)
accuracy_score(df['class'],pred)

0.9729600382866714